In [1]:
import numpy as np
import pandas as pd 
import torch
from glob import glob
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as T
import utils

In [2]:
DATA_DIR = "/home/will/Documents/datasets/cityscapes-image-pairs/cityscapes_data"
IMG_SIZE =256
train_path = glob(f'{DATA_DIR}/train/*')
valid_path = glob(f'{DATA_DIR}/val/*')

In [ ]:
fig, ax = plt.subplots(5,2,figsize=(10,30))
for i in range(3):
    img = plt.imread(train_path[i])
    ax[i][0].imshow(img[:,:256])
    ax[i][1].imshow(img[:,256:])

In [4]:
transformImage = T.Compose(
    [
        T.ToTensor(),
        T.Resize((IMG_SIZE, IMG_SIZE))
    ]
)

### Cityscapes

In [ ]:

train_data = utils.AnyDataset(train_path, transformImage, transformImage)
val_data = utils.AnyDataset(valid_path, transformImage, transformImage)

### Carla

In [ ]:
DATA_DIR = "/home/will/Documents/datasets/carla"
images = glob(f'{DATA_DIR}/images/*')
labels = glob(f'{DATA_DIR}/labels/*')

full_dataset = utils.AnyDataset(images, labels, transformImage, transformImage)
split = int(len(full_dataset) * 0.7)
train_data, val_data = torch.utils.data.random_split(
    full_dataset, [split, len(full_dataset) - split])

In [ ]:
from model import UNet
from torch.utils.data import DataLoader

lr = 0.01
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = UNet(3).float().to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_info = utils.train(
    model, train_data, val_data, loss_function, optimizer, epochs=30,
    show_every=15
)

In [7]:
import os
import pandas as pd
cwd = os.getcwd()

def save_results(results, name):
    pd.DataFrame(train_info[0]).to_csv(
        f"{cwd}/results/{name}/{name}_train.txt")

    pd.DataFrame(train_info[0]).to_csv(
        f"{cwd}/results/{name}/{name}_val.txt")

save_results(train_info, "carla")

In [ ]:
import matplotlib.pyplot as plt 

def plot_results(train_loss, val_loss):
    # fig, axs = plt.plot(figsize=(24, 8))
    # axs.plot()
    plt.figure(figsize=(10, 8))
    plt.plot(train_loss,color='b',label='train loss')
    plt.plot(val_loss,color='r',label = 'val_loss')
    plt.legend()
plot_results(*train_info)